In [6]:
import pandas as pd
import numpy as np

# Reload the datasets
final_hcris_v1996 = pd.read_csv('/Users/ryanscholte/Desktop/GitHub/HW2/data/output/Final_HCRIS_v1996.csv')
final_hcris_v2010 = pd.read_csv('/Users/ryanscholte/Desktop/GitHub/HW2/data/output/final_HCRIS_v2010.csv')

print(final_hcris_v1996.describe())




              report  provider_number  npi      status    year  beds  \
count     578.000000       578.000000  0.0  578.000000   578.0   0.0   
mean   288055.058824    343729.676471  NaN    1.970588  2011.0   NaN   
std     12151.908636    201232.500864  NaN    0.295841     0.0   NaN   
min    268339.000000     10016.000000  NaN    1.000000  2011.0   NaN   
25%    277013.000000    200032.000000  NaN    2.000000  2011.0   NaN   
50%    289367.500000    361583.500000  NaN    2.000000  2011.0   NaN   
75%    300457.000000    463302.000000  NaN    2.000000  2011.0   NaN   
max    302903.000000    673046.000000  NaN    3.000000  2011.0   NaN   

       tot_charges  tot_discounts  tot_operating_exp  ip_charges  ...  \
count          0.0            0.0                0.0         0.0  ...   
mean           NaN            NaN                NaN         NaN  ...   
std            NaN            NaN                NaN         NaN  ...   
min            NaN            NaN                NaN       

In [4]:
# Ensure column consistency between the two datasets
final_hcris_v1996['hvbp_payment'] = np.nan
final_hcris_v1996['hrrp_payment'] = np.nan

# Combine the datasets
final_hcris = pd.concat([final_hcris_v1996, final_hcris_v2010], ignore_index=True)

#print first 5 columns
print(final_hcris.describe())

              report  provider_number  npi        status          year  beds  \
count   67027.000000     67027.000000  0.0  67027.000000  67027.000000   0.0   
mean   601169.156549    270143.805407  NaN      1.823623   2015.472183   NaN   
std    104851.645084    158238.777000  NaN      1.311659      3.152153   NaN   
min       235.000000     10001.000000  NaN      1.000000   2010.000000   NaN   
25%    545533.500000    140211.000000  NaN      1.000000   2013.000000   NaN   
50%    592982.000000    260070.000000  NaN      1.000000   2015.000000   NaN   
75%    685090.500000    393040.000000  NaN      2.000000   2018.000000   NaN   
max    720558.000000    713026.000000  NaN      5.000000   2021.000000   NaN   

       tot_charges  tot_discounts  tot_operating_exp  ip_charges  ...  \
count          0.0            0.0                0.0         0.0  ...   
mean           NaN            NaN                NaN         NaN  ...   
std            NaN            NaN                NaN        

In [ ]:
# Convert date columns to datetime format
for col in ['fy_end', 'fy_start', 'date_processed', 'date_created']:
    if col in final_hcris.columns:
        final_hcris[col] = pd.to_datetime(final_hcris[col], errors='coerce')

# Convert to absolute values where necessary
for col in ['tot_discounts', 'hrrp_payment']:
    if col in final_hcris.columns:
        final_hcris[col] = final_hcris[col].abs()

# Extract fiscal year
if 'fy_end' in final_hcris.columns:
    final_hcris['year'] = final_hcris['fy_end'].dt.year  # Ensure this is created only once



In [ ]:
# Remove rows with missing critical identifiers
final_hcris = final_hcris.dropna(subset=['provider_number', 'year'])

# Ensure numeric columns do not contain NaN by filling with zeros
numeric_columns = ['tot_charges', 'tot_discounts', 'tot_operating_exp', 
                   'ip_charges', 'icu_charges', 'ancillary_charges', 
                   'tot_discharges', 'mcare_discharges', 'mcaid_discharges', 
                   'tot_mcare_payment', 'secondary_mcare_payment', 
                   'hvbp_payment', 'hrrp_payment']

for col in numeric_columns:
    if col in final_hcris.columns:
        final_hcris[col] = pd.to_numeric(final_hcris[col], errors='coerce').fillna(0)

# Create count of reports by hospital fiscal year
final_hcris['total_reports'] = final_hcris.groupby(['provider_number', 'year'])['provider_number'].transform('count')
final_hcris['report_number'] = final_hcris.groupby(['provider_number', 'year']).cumcount() + 1



In [ ]:
# Separate hospitals with one report per year
unique_hcris1 = final_hcris[final_hcris['total_reports'] == 1].drop(columns=['total_reports', 'report_number'], errors='ignore')
unique_hcris1['source'] = 'unique reports'

# Hospitals with multiple reports
duplicate_hcris = final_hcris[final_hcris['total_reports'] > 1].copy()

# Calculate elapsed time between fy start and fy end
duplicate_hcris['time_diff'] = (duplicate_hcris['fy_end'] - duplicate_hcris['fy_start']).dt.days
duplicate_hcris['total_days'] = duplicate_hcris.groupby(['provider_number', 'year'])['time_diff'].transform('sum')

# Aggregate hospitals where total days < 370
unique_hcris2 = duplicate_hcris[duplicate_hcris['total_days'] < 370].groupby(['provider_number', 'year']).agg({
    'beds': 'max', 'tot_charges': 'sum', 'tot_discounts': 'sum',
    'tot_operating_exp': 'sum', 'ip_charges': 'sum', 'icu_charges': 'sum',
    'ancillary_charges': 'sum', 'tot_discharges': 'sum', 'mcare_discharges': 'sum',
    'mcaid_discharges': 'sum', 'tot_mcare_payment': 'sum',
    'secondary_mcare_payment': 'sum', 'hvbp_payment': 'sum', 'hrrp_payment': 'sum',
    'fy_start': 'min', 'fy_end': 'max', 'date_processed': 'max', 'date_created': 'min',
    'street': 'first', 'city': 'first', 'state': 'first',
    'zip': 'first', 'county': 'first'
}).reset_index()
unique_hcris2['source'] = 'total for year'



In [ ]:
# Hospitals with reports exceeding 370 days
duplicate_hcris2 = duplicate_hcris[duplicate_hcris['total_days'] >= 370].copy()
duplicate_hcris2['max_days'] = duplicate_hcris2.groupby(['provider_number', 'year'])['time_diff'].transform('max')
duplicate_hcris2['max_date'] = duplicate_hcris2.groupby(['provider_number', 'year'])['fy_end'].transform('max')



In [ ]:
# Primary report selection
unique_hcris3 = duplicate_hcris2[(duplicate_hcris2['max_days'] == duplicate_hcris2['time_diff']) &
                                  (duplicate_hcris2['time_diff'] > 360) &
                                  (duplicate_hcris2['max_date'] == duplicate_hcris2['fy_end'])]
unique_hcris3 = unique_hcris3.drop(columns=['max_days', 'time_diff', 'total_days', 'max_date'], errors='ignore')
unique_hcris3['source'] = 'primary report'

# Remaining hospitals with reports covering more than one full year
duplicate_hcris3 = duplicate_hcris2[~duplicate_hcris2.index.isin(unique_hcris3.index)].copy()
duplicate_hcris3['time_diff'] = duplicate_hcris3['time_diff'].astype(int)
duplicate_hcris3['total_days'] = duplicate_hcris3['total_days'].astype(int)

# Apply weighted adjustment
duplicate_hcris3[numeric_columns] = duplicate_hcris3[numeric_columns].mul(
    duplicate_hcris3['time_diff'] / duplicate_hcris3['total_days'], axis=0)



In [ ]:
# Aggregate weighted hospitals
unique_hcris4 = duplicate_hcris3.groupby(['provider_number', 'year']).agg({
    'beds': 'max', 'tot_charges': 'sum', 'tot_discounts': 'sum',
    'tot_operating_exp': 'sum', 'ip_charges': 'sum', 'icu_charges': 'sum',
    'ancillary_charges': 'sum', 'tot_discharges': 'sum', 'mcare_discharges': 'sum',
    'mcaid_discharges': 'sum', 'tot_mcare_payment': 'sum',
    'secondary_mcare_payment': 'sum', 'hvbp_payment': 'sum', 'hrrp_payment': 'sum',
    'fy_start': 'min', 'fy_end': 'max', 'date_processed': 'max', 'date_created': 'min',
    'street': 'first', 'city': 'first', 'state': 'first',
    'zip': 'first', 'county': 'first'
}).reset_index()
unique_hcris4['source'] = 'weighted_average'


In [ ]:

# Combine cleaned datasets
final_hcris_data = pd.concat([unique_hcris1, unique_hcris2, unique_hcris3, unique_hcris4], ignore_index=True)

# Ensure no duplicate columns before sorting
final_hcris_data = final_hcris_data.loc[:, ~final_hcris_data.columns.duplicated()]

# Sort the final dataset
final_hcris_data = final_hcris_data.sort_values(by=['provider_number', 'year'])